# Build & Run & Read

In [1]:
# Change directory to the root folder of Dispa-SET:
import os
import sys
os.chdir('..')
import numpy as np
import pandas as pd
import datetime as dt
from dispaset.postprocessing import postprocessing as post
import importlib
importlib.reload(post)
%matplotlib notebook
#Import Dispa-SET
import dispaset as ds
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import seaborn as sns
import matplotlib.dates as mdates
#plt.style.use('seaborn-colorblind')
import pickle


In [2]:
# Load the configuration file
config = ds.load_config_excel('Simulations/ConfigGCC.xlsx')

[INFO    ] (load_config_excel): Using config file Simulations/ConfigGCC.xlsx to build the simulation environment
[INFO    ] (load_config_excel): Using /Users/abdullahalawad/Dropbox (MIT)/CCES/2017/Energy/Regional trade/UC&ED/dispaset/PycharmProjects/UC Model/Simulations/simulation_GCC as simulation folder


In [3]:
config['SimulationDirectory'] = u'Simulations/Test7'
config['StartDate'] = (2016, 7, 11, 0, 0, 0)      # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config['StopDate'] = (2016, 7, 17, 0, 0, 0)      # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config['HorizonLength'] = 7
config['LookAhead'] = 1
config['SimulationType'] = u'Integer clustering'               # simulation type (LP or MILP) and clustering type (Integer or LP clustering)
config['ReserveCalculation'] = u'Generic'
config['AllowCurtailment'] = 1                                 #1 for True & 0 for False
config['GAMS_ModelCode'] = 'GCC_virtual_connections'           #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

In [4]:
#NTCs for not considering fictitious nodes (e.g. Alfadhili). NTCs2 for considering them
#NTCs_isolated & NTCs2_isolated for isolated countries Simulation. 

#config['NTC'] = u'Database/GCCData/NTCs2.csv'     

#config['NTC'] = u'Database/GCCData/NTCs.csv'
#config['zones'] = [x for x in config['zones'] if x not in ['Alfadhili','Ghunan','Salwa']]  #drop fictitious nodes

In [5]:
config['PriceOfCrudeOil'] = None 
config['PriceOfDiesel'] = None 
config['PriceOfGas'] = None 
config['PriceOfHFO'] = None 

#config['PriceOfCrudeOil'] = u'Database/FuelPrices/Oil/2016_int.csv'
#config['PriceOfDiesel'] = u'Database/FuelPrices/Diesel/2016_int.csv'
#config['PriceOfGas'] = u'Database/FuelPrices/Gas/2016_int.csv'
#config['PriceOfHFO'] = u'Database/FuelPrices/HFO/2016_int.csv'

In [6]:
config['PriceOfCrudeOil 2'] = None 
config['PriceOfDiesel 2'] = None 
config['PriceOfGas 2'] = None 
config['PriceOfHFO 2'] = None 

#config['PriceOfCrudeOil 2'] = u'Database/FuelPrices/Oil/2016_sub.csv'
#config['PriceOfDiesel 2'] = u'Database/FuelPrices/Diesel/2016_sub.csv'
#config['PriceOfGas 2'] = u'Database/FuelPrices/Gas/2016_sub.csv'
#config['PriceOfHFO 2'] = u'Database/FuelPrices/HFO/2016_sub.csv'

In [ ]:
# Build the simulation environment:
SimData, FuelPrices, FuelPrices2 = ds.build_simulation(config, LocalSubsidyMultiplier=1, ExportCostMultiplier=1)

# Simulation loop

In [45]:
config = ds.load_config_excel('Simulations/ConfigGCC.xlsx')
config['SimulationDirectory'] = u'Simulations/Test7'
config['StartDate'] = (2016, 7, 11, 0, 0, 0)          # '1/1/16'  (2016, 1, 1, 0, 0, 0)
config['StopDate'] = (2016, 7, 17, 0, 0, 0)           # '12/31/16'  (2016, 12, 31, 0, 0, 0)
config['HorizonLength'] = 7
config['LookAhead'] = 1
config['SimulationType'] = u'Integer clustering'      # simulation type (LP or MILP) and clustering type (Integer or LP clustering)         
config['ReserveCalculation'] = u'Generic'
config['AllowCurtailment'] = 1                        #1 for True & 0 for False
config['PriceOfCrudeOil'] = None
config['PriceOfDiesel'] = None
config['PriceOfGas'] = None
config['PriceOfHFO'] = None
config['PriceOfCrudeOil 2'] = None
config['PriceOfDiesel 2'] = None
config['PriceOfGas 2'] = None
config['PriceOfHFO 2'] = None
#config['GAMS_folder'] = '/Applications/GAMS26.1/sysdir/'
#config['GAMS_folder'] = 'C:\GAMS\win64\24.7'
config['GAMS_ModelCode'] == 'GCC_virtual_connections'  #'Standard', 'GCC_virtual_connections', 'GCC_substation_nodes', 'GCC_isolated'

Results_dic = {}
Inputs_dic = {}
FuelPrices_dic = {}
for i in np.arange(1, 0, -0.02):
    print(i)
    SimData, FuelPrices, FuelPrices2 = ds.build_simulation(config, LocalSubsidyMultiplier=i, ExportCostMultiplier=1)
    LocalSubsidyMultiplier=i
    ExportCostMultiplier=1
    
    r = ds.solve_GAMS(config['SimulationDirectory'], config['GAMS_folder'])
    
    path = 'Simulations/Test7' 
    inputs,results = ds.get_sim_results(path=path,cache=True)
    inputs = ds.ds_to_df(inputs)       #convert input data format into a dictionary of dataframes
    
    Results_dic['LocalSubsidyMultiplier: '+str(LocalSubsidyMultiplier)+' & ExportCostMultiplier: '+str(ExportCostMultiplier)] = results

    Inputs_dic['LocalSubsidyMultiplier: '+str(LocalSubsidyMultiplier)+' & ExportCostMultiplier: '+str(ExportCostMultiplier)] = inputs

    FuelPrices_dic['LocalSubsidyMultiplier: '+str(LocalSubsidyMultiplier)+' & ExportCostMultiplier: '+str(ExportCostMultiplier)+' & FuelPrices 1'] = FuelPrices
    FuelPrices_dic['LocalSubsidyMultiplier: '+str(LocalSubsidyMultiplier)+' & ExportCostMultiplier: '+str(ExportCostMultiplier)+' & FuelPrices 2'] = FuelPrices2


[INFO    ] (load_config_excel): Using config file Simulations/ConfigGCC.xlsx to build the simulation environment
[INFO    ] (load_config_excel): Using /Users/abdullahalawad/Dropbox (MIT)/CCES/2017/Energy/Regional trade/UC&ED/dispaset/PycharmProjects/UC Model/Simulations/simulation_GCC as simulation folder
[INFO    ] (build_simulation): New build started. DispaSET version: b'v2.3-8-g65fd75a'
[INFO    ] (NodeBasedTable): No data file found for the table LoadShedding. Using default value 0.0
[INFO    ] (NodeBasedTable): No data file found for the table CostLoadShedding. Using default value 1000.0
[INFO    ] (check_units): The columns "Nunits" is not present in the power plant database. A value of one will be assumed by default


1.0


[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone UAE. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 1018 original units into 67 new units
[INFO    ] (build_simulation): Time to cluster power plants: 39.703460931777954s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (build_simulation): TimeDownMinimum in the power plant data has been rounded to the nearest integer value
[INFO    ] (build_simulation): Scaling Storage Power and Capacity by a factor 0.0
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (write_variables): Data Successfully writte

0.98


[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone UAE. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 1018 original units into 67 new units
[INFO    ] (build_simulation): Time to cluster power plants: 37.94655990600586s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (build_simulation): TimeDownMinimum in the power plant data has been rounded to the nearest integer value
[INFO    ] (build_simulation): Scaling Storage Power and Capacity by a factor 0.0
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (write_variables): Data Successfully written

...

[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone UAE. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 1018 original units into 67 new units
[INFO    ] (build_simulation): Time to cluster power plants: 38.14108324050903s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (build_simulation): TimeDownMinimum in the power plant data has been rounded to the nearest integer value
[INFO    ] (build_simulation): Scaling Storage Power and Capacity by a factor 0.0
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (write_variables): Data Successfully written

0.03999999999999915


[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone UAE. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 1018 original units into 67 new units
[INFO    ] (build_simulation): Time to cluster power plants: 38.011274099349976s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (build_simulation): TimeDownMinimum in the power plant data has been rounded to the nearest integer value
[INFO    ] (build_simulation): Scaling Storage Power and Capacity by a factor 0.0
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (write_variables): Data Successfully writte

0.01999999999999913


[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone UAE. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 1018 original units into 67 new units
[INFO    ] (build_simulation): Time to cluster power plants: 38.03860020637512s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (build_simulation): TimeDownMinimum in the power plant data has been rounded to the nearest integer value
[INFO    ] (build_simulation): Scaling Storage Power and Capacity by a factor 0.0
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (write_variables): Data Successfully written

In [12]:
Results_dic.keys()


dict_keys(['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.98 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.96 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.94 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.9199999999999999 & ExportCostMultiplier: 1'])

In [16]:
Results_dic['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1'].keys()


dict_keys(['*', 'UnitCommitment', 'PowerFlow', 'TotalNodeOperationCost', 'TotalNodeVariableCost', 'TotalKSAVariableCost', 'NetNodeVariableCost', 'NetKSAVariableCost', 'ElectricityNodePrice', 'ElectricityNodePrice2', 'ElectricityNodePrice3', 'ElectricityKSAPrice', 'ElectricityKSAPrice2', 'ElectricityKSAPrice3', 'NodeOutputCurtailedPower', 'NodeShadowPrice', 'TotalNodeDemand', 'TotalKSADemand', 'status', 'PowerFlowMaxLimit', 'UnitOutputPower', 'UnitOutputPowerInside', 'UnitOutputPowerOutside', 'UnitOutputPowerForNode', 'UnitStartUpCost', 'UnitVariableCost', 'UnitOperationCost', 'LocalOutputPower', 'LocalOutputPowerCost', 'KSALocalOutputPower', 'KSALocalOutputPowerCost', 'TotalImportedPower', 'TotalImportedPowerCost', 'NetImportedPower', 'NetImportedPowerCost', 'ImportedPowerFromNode', 'ImportedPowerFromNodeCost', 'NetImportedPowerFromNode', 'NetImportedPowerFromNodeCost', 'KSATotalImportedPower', 'KSATotalImportedPowerCost', 'KSANetImportedPower', 'KSANetImportedPowerCost', 'KSAImportedP

In [17]:
Results_dic['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1']['PowerFlow']


,BA -> SA_EOA,SA_EOA -> BA,KW -> SA_EOA,SA_EOA -> KW,UAE -> SA_EOA,SA_EOA -> UAE,QA -> SA_EOA,BA -> QA,QA -> UAE,UAE -> BA,...,OM -> UAE,UAE -> OM,SA_RIYADH -> SA_EOA,SA_EOA -> SA_RIYADH,SA_RIYADH -> SA_QASSIM,SA_WOA -> SA_QASSIM,SA_HAIL -> SA_NEOA,SA_QASSIM -> SA_HAIL,SA_SOA -> SA_WOA,SA_WOA -> SA_SOA
2016-07-11 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.366220,0.000000,2262.936277,2752.644539,328.195710,800.000000,913.451674,1158.772326,0.000000
2016-07-11 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.347636,0.000000,679.537246,2424.481710,624.509539,800.000000,903.831674,900.147754,0.000000
2016-07-11 02:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.366216,0.000000,688.893246,2572.146710,440.866539,800.000000,892.956674,699.410754,0.000000
2016-07-11 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.373361,0.000000,668.597330,2710.450794,261.285455,800.000000,880.595674,313.537671,0.000000
2016-07-11 04:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.369071,0.000000,766.223330,2882.937794,69.318455,800.000000,874.746674,198.560671,0.000000
2016-07-11 05:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.367640,0.000000,1054.793246,2909.453710,110.996539,800.000000,895.210674,640.153754,0.000000
2016-07-11 06:00:00,0.000000,0.503921,0.115137,0.000000,0.000000,0.000000,0.000000,0.564876,0.585727,0.000000,...,0.000000,6.917068,0.000000,405.389440,2578.023447,359.701802,800.000000,870.405674,761.473628,0.000000
2016-07-11 07:00:00,0.000000,1.612549,0.368439,0.000000,0.000000,0.000000,0.000000,1.807604,1.874327,0.000000,...,0.000000,16.904228,0.000000,958.156879,2485.457038,517.399211,800.000000,889.909674,858.117180,0.000000
2016-07-11 08:00:00,0.000000,2.838757,0.648606,0.000000,0.000000,0.000000,0.000000,3.182137,3.299596,0.000000,...,0.000000,27.924170,0.000000,629.678970,2263.535919,801.014330,800.000000,908.487674,709.154901,0.000000
2016-07-11 09:00:00,0.000000,4.350522,0.994017,0.000000,0.000000,0.000000,0.000000,4.876766,5.056777,0.000000,...,0.000000,41.561739,0.000000,719.445198,2038.979618,1120.885630,800.000000,937.300674,1409.804033,0.000000


In [46]:
Inputs_dic.keys()


dict_keys(['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.98 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.96 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.94 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.9199999999999999 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.8999999999999999 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.8799999999999999 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.8599999999999999 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.8399999999999999 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.8199999999999998 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.7999999999999998 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.7799999999999998 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.7599999999999998 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.7399999999999998 & ExportCostMultiplier: 1', 'LocalSubsidyMultiplier: 0.71999999999

In [48]:
Inputs_dic['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1'].keys()

dict_keys(['sets', 'AvailabilityFactor', 'CHPPowerToHeat', 'CHPPowerLossFactor', 'CHPMaxHeat', 'CostFixed', 'CostHeatSlack', 'CostLoadShedding', 'CostRampUp', 'CostRampDown', 'CostShutDown', 'CostStartUp', 'CostVariable', 'CostVariableB', 'Curtailment', 'Demand', 'Efficiency', 'EmissionMaximum', 'EmissionRate', 'FlowMaximum', 'FlowMinimum', 'FuelPrice', 'Fuel', 'HeatDemand', 'LineNode', 'LoadShedding', 'Location', 'Markup', 'Nunits', 'OutageFactor', 'PartLoadMin', 'PowerCapacity', 'PowerInitial', 'PriceTransmission', 'RampUpMaximum', 'RampDownMaximum', 'RampStartUpMaximum', 'RampShutDownMaximum', 'Reserve', 'StorageCapacity', 'StorageChargingCapacity', 'StorageChargingEfficiency', 'StorageDischargeEfficiency', 'StorageSelfDischarge', 'StorageInflow', 'StorageInitial', 'StorageMinimum', 'StorageOutflow', 'StorageProfile', 'Technology', 'TimeUpMinimum', 'TimeDownMinimum', 'Config', 'FuelPricePerZone'])

In [50]:
Inputs_dic['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1']['PowerCapacity']

,PowerCapacity
"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] - SA_EOA - STUR - GAS -",573.000000
"[12, 13, 14, 15, 16, ... _EOA - GTUR - GAS -",70.372260
"[30, 31, 32, 33, 34, 35] - SA_EOA - COMC - GAS -",665.000000
"[58, 59, 60, 61, 62, 63] - SA_EOA - GTUR - DSL -",20.266667
"[105, 106, 107, 108, ... _EOA - ICEN - GAS -",1462.500000
[113] - Saudi Aramco North Park Project,10.500000
"[114, 115, 116, 117, ... HAIL - GTUR - OIL -",70.884042
"[126, 127, 128, 129, ... NEOA - GTUR - OIL -",34.300000
"[136, 137, 138, 139, ... NEOA - GTUR - DSL -",36.900000
"[146, 151] - SA_NWOA - GTUR - DSL -",302.000000


In [14]:
FuelPrices_dic.keys()


dict_keys(['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1 & FuelPrices 1', 'LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1 & FuelPrices 2', 'LocalSubsidyMultiplier: 0.98 & ExportCostMultiplier: 1 & FuelPrices 1', 'LocalSubsidyMultiplier: 0.98 & ExportCostMultiplier: 1 & FuelPrices 2', 'LocalSubsidyMultiplier: 0.96 & ExportCostMultiplier: 1 & FuelPrices 1', 'LocalSubsidyMultiplier: 0.96 & ExportCostMultiplier: 1 & FuelPrices 2', 'LocalSubsidyMultiplier: 0.94 & ExportCostMultiplier: 1 & FuelPrices 1', 'LocalSubsidyMultiplier: 0.94 & ExportCostMultiplier: 1 & FuelPrices 2', 'LocalSubsidyMultiplier: 0.9199999999999999 & ExportCostMultiplier: 1 & FuelPrices 1', 'LocalSubsidyMultiplier: 0.9199999999999999 & ExportCostMultiplier: 1 & FuelPrices 2'])

In [26]:
FuelPrices_dic['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1 & FuelPrices 1']

,SA_EOA,SA_RIYADH,SA_QASSIM,SA_HAIL,SA_WOA,SA_SOA,SA_NWOA,SA_NEOA,OM,KW,...,SA_HAIL,SA_WOA,SA_SOA,SA_NWOA,SA_NEOA,OM,KW,BA,QA,UAE
,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,PriceOfGas,...,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas,PriceOfLandFillGas
2016-07-10 23:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 00:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 01:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 02:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 03:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 04:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 05:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 06:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596
2016-07-11 07:00:00,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,2.557133,10.228532,2.557133,...,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596,40.936596


In [51]:
FuelPrices_dic['LocalSubsidyMultiplier: 1.0 & ExportCostMultiplier: 1 & FuelPrices 2']

,SA_EOA,SA_RIYADH,SA_QASSIM,SA_HAIL,SA_WOA,SA_SOA,SA_NWOA,SA_NEOA,OM,KW,...,SA_HAIL,SA_WOA,SA_SOA,SA_NWOA,SA_NEOA,OM,KW,BA,QA,UAE
,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,PriceOfNuclear 2,...,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2,PriceOfBlackCoal 2
2016-07-10 23:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 00:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 01:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 02:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 03:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 04:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 05:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 06:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0
2016-07-11 07:00:00,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,3.319183,...,0,0,0,0,0,0,0,0,0,0


## To be read:

In [ ]:
path = '../Simulations/Test7' 
inputs,results = ds.get_sim_results(path=path,cache=True)
datain = ds.ds_to_df(inputs)

SystemCost = results['TotalSystemCost'].sum()*(1/1000000)     #Million dollars

NodeCost = results['TotalNodeOperationCost'].copy() 
NodeCost['KSA'] = NodeCost['SA_EOA']+NodeCost['SA_HAIL']+NodeCost['SA_NEOA']+NodeCost['SA_NWOA']+NodeCost['SA_QASSIM']+NodeCost['SA_RIYADH']+NodeCost['SA_SOA']+NodeCost['SA_WOA']
NodeCost = NodeCost.sum()/(1000000)   #Million dollars

#Average electricity cost in $/MWh 
A = pd.Series((datain['Demand']['DA'].sum().drop(['BA','KW','OM','QA','UAE'])).sum())
A.name = 'KSA'
System_demand = pd.DataFrame(datain['Demand']['DA'].sum())
System_demand = System_demand.append(A, ignore_index=False)
System_demand.columns = ['TotalDemand']
elec_cost = pd.Series(SystemCost['SystemCost']*(1000000), index=SystemCost.index)/pd.Series(System_demand['TotalDemand'], index=System_demand.index)
elec_cost


NewOutputFlow = results['PowerFlow'].copy()
columns = []
for i,column in enumerate(NewOutputFlow):
    [from_node, to_node] = column.split('->')
    if (from_node.strip()[0:2] == 'SA' and to_node.strip()[0:2] == 'SA'):
        columns.append(i)
GCCFlows = NewOutputFlow.drop(NewOutputFlow.columns[columns], axis=1).copy()
CrossBorderPowerFlow = GCCFlows.sum().sum()   #MWh

results['TotalExportedPower'].sum()

results['NodeShadowPrice'].mean()
results['NodeShadowPrice'].std()
results['NodeShadowPrice'].mad()

PPindicators = post.get_indicators_powerplant(inputs,results)

fuels = PPindicators.Fuel.unique()
countries = PPindicators.Zone.unique()

GenPerCountry = pd.DataFrame(index=countries, columns=fuels)
# First make sure that all fuels are present. If not, initialize an empty series
for f in ['NUC', 'LFG', 'DSL', 'MSW', 'GAS', 'OIL', 'WST', 'SUN', 'WIN', 'FlowIn', 'HFO']:
    if f not in GenPerCountry:
        GenPerCountry[f] = 0
for c in countries:
    for f in fuels:
        tmp = PPindicators[(PPindicators.Fuel == f) & (PPindicators.Zone == c)]
        GenPerCountry.loc[c, f] = tmp.Generation.sum()
    NetImports = post.get_imports(results['PowerFlow'], c)
    if NetImports > 0:
        GenPerCountry.loc[c, 'FlowIn'] = NetImports
GenPerCountry = GenPerCountry.copy()
A = GenPerCountry.drop(['BA','KW','OM','QA','UAE']).sum()
A.name = 'KSA'
GenPerCountry = GenPerCountry.append(A, ignore_index=False)

